In [1]:
from Pegasus.api import *

In [2]:
# not required. Just to view output from certain tools.
from pathlib import Path
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
# creating input file number.txt 
with open("number.txt", "w") as f:
    f.write("10")

number = File("number.txt").add_metadata(creator="shubham")

In [7]:
rc = ReplicaCatalog()\
    .add_replica("local", number, Path(".").resolve() / "number.txt")
rc.write()

tc = TransformationCatalog()

increment = Transformation(
                "increment",
                site="local",
                pfn="/home/scitech/shared-data/increment.py",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
)

tc.add_transformations(increment)
tc.write()

In [8]:
# create workflow

wf = Workflow("Pipeline")

result = File("result.txt")

job_increment = Job(increment)\
                .add_args(number, result)\
                .add_inputs(number)\
                .add_outputs(result)
wf.add_jobs(job_increment)

In [10]:
try:
    wf.plan(submit=True)\
        .wait()
except PegasusClientError as e:
    print(e)



################
# pegasus-plan #
################
2021.01.28 17:40:07.389 UTC:
2021.01.28 17:40:07.395 UTC:   -----------------------------------------------------------------------
2021.01.28 17:40:07.403 UTC:   File for submitting this DAG to HTCondor           : Pipeline-0.dag.condor.sub
2021.01.28 17:40:07.411 UTC:   Log of DAGMan debugging messages                 : Pipeline-0.dag.dagman.out
2021.01.28 17:40:07.419 UTC:   Log of HTCondor library output                     : Pipeline-0.dag.lib.out
2021.01.28 17:40:07.429 UTC:   Log of HTCondor library error messages             : Pipeline-0.dag.lib.err
2021.01.28 17:40:07.437 UTC:   Log of the life of condor_dagman itself          : Pipeline-0.dag.dagman.log
2021.01.28 17:40:07.444 UTC:
2021.01.28 17:40:07.450 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with:
2021.01.28 17:40:07.462 UTC:   -----------------------------------------------------------------------
2021.01.28 17:40:08.754 UTC:   Your data

[####################################] 100.0% ..Success (Completed: 9, Queued: 0, Running: 0, Failed: 0)
